In [1]:
import json
import os
import sys
import getpass
import hashlib
import uuid
from datetime import datetime
from typing import Dict, Any, List

In [2]:
DATA_FILE = "lms_data.json"

In [ ]:
# Utilities
def now_str():
    return datetime.now().isoformat(timespec="seconds")


def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()


def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")


def pause(msg="Press Enter to continue..."):
    input(msg)

In [4]:
# Data model helpers

def load_data():
    if not os.path.exists(DATA_FILE):
        return {"users": {}, "courses": {}}
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


def save_data(data):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)


In [6]:
# User and Auth
def create_user(data, username, password, role, full_name=""):
    if username in data["users"]:
        return False, "username already exists"
    uid = str(uuid.uuid4())
    data["users"][username] = {
        "id": uid,
        "username": username,
        "full_name": full_name,
        "password_hash": hash_password(password),
        "role": role,
        "enrolled": [],         # list of course_ids for students
        "created_courses": [],  # list of course_ids for teachers
        "grades": {}            # {course_id: {quiz_id: score}}
    }
    save_data(data)
    return True, "user created"


def authenticate(data, username, password):
    user = data["users"].get(username)
    if not user:
        return None
    if user["password_hash"] == hash_password(password):
        return user
    return None


In [8]:
# Course & content management
def new_course(data, creator_username, title, description):
    course_id = str(uuid.uuid4())
    data["courses"][course_id] = {
        "id": course_id,
        "title": title,
        "description": description,
        "created_by": creator_username,
        "created_at": now_str(),
        "lessons": [],  # list of {id, title, content}
        "quizzes": []   # list of {id, title, questions:[{q, options, answer_index}]}
    }
    data["users"][creator_username]["created_courses"].append(course_id)
    save_data(data)
    return course_id


def add_lesson(data, course_id, title, content):
    lesson_id = str(uuid.uuid4())
    lesson = {"id": lesson_id, "title": title, "content": content, "created_at": now_str()}
    data["courses"][course_id]["lessons"].append(lesson)
    save_data(data)
    return lesson_id


def add_quiz(data, course_id, title, questions):
    """
    questions: list of dicts {q: str, options: [str], answer_index: int}
    """
    quiz_id = str(uuid.uuid4())
    quiz = {"id": quiz_id, "title": title, "questions": questions, "created_at": now_str()}
    data["courses"][course_id]["quizzes"].append(quiz)
    save_data(data)
    return quiz_id

def enroll_student(data, username, course_id):
    if course_id in data["users"][username]["enrolled"]:
        return False, "Already enrolled"
    data["users"][username]["enrolled"].append(course_id)
    save_data(data)
    return True, "Enrolled successfully"


def record_grade(data, username, course_id, quiz_id, score, out_of):
    grades = data["users"][username].setdefault("grades", {})
    course_grades = grades.setdefault(course_id, {})
    course_grades[quiz_id] = {"score": score, "out_of": out_of, "ts": now_str()}
    save_data(data)



In [9]:
# Terminal UI: menus

def main_menu():
    clear_screen()
    print("=== Terminal LMS ===")
    print("1) Login")
    print("2) Register (student)")
    print("3) Exit")
    choice = input("Choose: ").strip()
    return choice


In [10]:
def admin_menu(user, data):
    while True:
        clear_screen()
        print(f"Admin: {user['username']} ({user.get('full_name','')})")
        print("1) Create user")
        print("2) List users")
        print("3) List courses")
        print("4) Logout")
        c = input("Choose: ").strip()
        if c == "1":
            uname = input("username: ").strip()
            pwd = getpass.getpass("password: ").strip()
            role = input("role (admin/teacher/student): ").strip().lower()
            full = input("full name: ").strip()
            ok, msg = create_user(data, uname, pwd, role, full)
            print(msg)
            pause()
        elif c == "2":
            print("Users:")
            for u in data["users"].values():
                print(f"- {u['username']} ({u['role']}) - {u.get('full_name','')}")
            pause()
        elif c == "3":
            list_all_courses(data)
            pause()
        elif c == "4":
            break
        else:
            print("Invalid choice"); pause()


In [11]:
def teacher_menu(user, data):
    while True:
        clear_screen()
        print(f"Teacher: {user['username']}")
        print("1) Create course")
        print("2) My courses")
        print("3) List all courses")
        print("4) Logout")
        c = input("Choose: ").strip()
        if c == "1":
            title = input("Course title: ").strip()
            desc = input("Description: ").strip()
            cid = new_course(data, user["username"], title, desc)
            print("Course created:", cid)
            pause()
        elif c == "2":
            my_courses = user.get("created_courses", [])
            if not my_courses:
                print("You haven't created any courses yet.")
                pause(); continue
            for i, cid in enumerate(my_courses, 1):
                course = data["courses"][cid]
                print(f"{i}) {course['title']} - {course['id']}")
            sel = input("Choose course number to manage (or Enter to go back): ").strip()
            if not sel:
                continue
            try:
                idx = int(sel) - 1
                cid = my_courses[idx]
                teacher_course_manage_menu(user, data, cid)
            except Exception as e:
                print("Invalid selection.", e)
                pause()
        elif c == "3":
            list_all_courses(data)
            pause()
        elif c == "4":
            break
        else:
            print("Invalid choice"); pause()



In [ ]:
def teacher_course_manage_menu(user, data, course_id):
    course = data["courses"][course_id]
    while True:
        clear_screen()
        print(f"Manage Course: {course['title']}")
        print("1) Add lesson")
        print("2) Add quiz")
        print("3) List lessons & quizzes")
        print("4) Back")
        c = input("Choose: ").strip()
        if c == "1":
            title = input("Lesson title: ").strip()
            print("Enter lesson content. Finish with a single line containing only 'END'.")
            lines = []
            while True:
                line = input()
                if line.strip() == "END":
                    break
                lines.append(line)
            content = "\n".join(lines)
            add_lesson(data, course_id, title, content)
            print("Lesson added.")
            pause()
        elif c == "2":
            qtitle = input("Quiz title: ").strip()
            questions = []
            qcount = input("How many questions? ").strip()
            try:
                qcount = int(qcount)
            except:
                qcount = 0
            for qi in range(qcount):
                print(f"Question {qi+1}:")
                qtext = input("Q: ").strip()
                opts = []
                for oi in range(4):
                    opt = input(f"Option {oi+1}: ").strip()
                    opts.append(opt)
                ans = input("Correct option number (1-4): ").strip()
                try:
                    ans_idx = int(ans) - 1
                    if not (0 <= ans_idx < len(opts)):
                        raise ValueError
                except:
                    ans_idx = 0
                questions.append({"q": qtext, "options": opts, "answer_index": ans_idx})
            add_quiz(data, course_id, qtitle, questions)
            print("Quiz added.")
            pause()
        elif c == "3":
            print("Lessons:")
            for L in course["lessons"]:
                print(f"- {L['title']} (id={L['id']})")
            print("\nQuizzes:")
            for Q in course["quizzes"]:
                print(f"- {Q['title']} (id={Q['id']}) - {len(Q['questions'])} questions")
            pause()
        elif c == "4":
            break
        else:
            print("Invalid"); pause()
